In [251]:
import pandas as pd
pd.options.display.max_columns = 60

### Ремонтные работы

In [252]:
def split_jobs(date, job):
    
    # функция для преобразования строк дата, работа в списки пар (дата (числом) + работа )
    
    if date.strip().isnumeric():
        return [(int(date.strip()), job), ]
    elif ',' in date:
        return [(int(d.strip(' ')), job) for d in date.split(',') if (d.strip(' ').isnumeric())]
    elif '/' in date:
        return [(int(d.strip(' ')), job) for d in date.split('/') if (d.strip(' ').isnumeric())]
    elif '-' in date:
        return [(int(date[-4:]), job), ]
    else:
        return [(0, '0'), ]

In [253]:
# оставим только релевантные колонки
df = pd.read_csv('data/housing_data.csv')
df_repair = df[['data_buildingdate', 'data_reconstructiondate', 'param_failure', 'repair_year', 'repair_job']]
df_repair.head()

,data_buildingdate,data_reconstructiondate,param_failure,repair_year,repair_job
0,1986,NaN,0.0,2014,Капитальный ремонт лифтов с частичной заменой ...
1,1966,NaN,0.0,2002; 2004; 2004; 2004; 2005; 2007; 2008; 2009...,Благоустройство придомовой территории; Благоус...
2,1968,NaN,0.0,2005; 2005; 2010; 2012; 2014,Капитальный ремонт лифтов с полной заменой обо...
3,1966,NaN,0.0,NaN,NaN
4,1974,NaN,0.0,2007; 2007; 2008,Прочие работы по благоустройству; Ремонт и вос...
...,...,...,...,...,...
22240,2002,NaN,0.0,NaN,NaN
22241,1900,NaN,0.0,2011; 2016,Ремонт или замена системы отопления; Ремонт ил...
22242,1961,NaN,0.0,1981; 2008; 2010; 2010,Разные работы; Разные работы; Ремонт или замен...
22243,1958,NaN,0.0,2010; 2010; 2010; 2011; 2011; 2011; 2011,Ремонт или замена системы горячего водоснабжен...


Составим датафрейм вида: дом-тип_работ-дата

In [254]:
# преобразуем строки со списками работ и дат в списки
df_repair = df_repair[['repair_year', 'repair_job']].dropna()
df_repair['repair_year'] = df['repair_year'].dropna().apply(lambda x: x.split('; '))
df_repair['repair_job'] = df['repair_job'].dropna().apply(lambda x: x.split('; '))
df_repair['pairs'] = df_repair.apply(lambda x: list(zip(x['repair_year'], x['repair_job'])), axis=1)

In [255]:
# разберем списки пар работа-дата на отдельные пары
df_repair_triplets = []
for index, row in zip(df_repair.index, df_repair['pairs']):
    row_converted = []
    for item in row:
        row_converted.extend(split_jobs(*item))
    for item in row_converted:
        try:
            df_repair_triplets.append([index, item[1], item[0]])
        except IndexError:
            print(item)

In [259]:
# преобразуем лист в датафрейм, уберем нереалистичные даты
df_repair_triplets = pd.DataFrame(data=df_repair_triplets, columns=['building', 'job', 'date'])
df_repair_triplets = df_repair_triplets.set_index('building')
df_repair_triplets = df_repair_triplets[(df_repair_triplets['date'] < 2020) & (df_repair_triplets['date'] > 1900)]
df_repair_triplets.to_csv('data/repairs.')

### Аварийное состояние

In [262]:
# оставим только релевантные колонки
df = pd.read_csv('data/housing_data.csv')
df_repair = df[['data_buildingdate', 'data_reconstructiondate', 'param_failure', 'repair_year', 'repair_job']]
df_repair.head()

,data_buildingdate,data_reconstructiondate,param_failure,repair_year,repair_job
0,1986,NaN,0.0,2014,Капитальный ремонт лифтов с частичной заменой ...
1,1966,NaN,0.0,2002; 2004; 2004; 2004; 2005; 2007; 2008; 2009...,Благоустройство придомовой территории; Благоус...
2,1968,NaN,0.0,2005; 2005; 2010; 2012; 2014,Капитальный ремонт лифтов с полной заменой обо...
3,1966,NaN,0.0,NaN,NaN
4,1974,NaN,0.0,2007; 2007; 2008,Прочие работы по благоустройству; Ремонт и вос...


In [263]:
df_repair['param_failure'].sum()

202.0

In [271]:
df_repair[df_repair['param_failure'] != 0][['param_failure',]]

,param_failure
46,1.0
328,1.0
450,1.0
452,1.0
573,1.0
...,...
21720,1.0
21919,1.0
22097,1.0
22197,1.0


In [272]:
df_repair[df_repair['param_failure'] != 0][['param_failure',]].to_csv('data/failure.csv')

### Реконструкции

In [269]:
df_repair[df_repair['data_reconstructiondate'].notna()][['data_reconstructiondate', ]]

,data_reconstructiondate
17,1985
18,2007
19,1970
24,1961
27,1964
...,...
22164,1947
22199,1974
22205,2004
22217,1969


In [273]:
df_repair[df_repair['data_reconstructiondate'].notna()][['data_reconstructiondate', ]].to_csv('data/reconstruction.csv')